In [17]:
from utils.card_engine import Card_Game, Card_Env, random_agent, get_legal_moves, net_policy, policy_legal_move

In [2]:
import math
import random
from collections import namedtuple, deque
import pickle

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import matplotlib
import matplotlib.pyplot as plt

from itertools import count

from tqdm import tqdm

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

# Replay Memory

The replay memory is a list of 13 double ended queues (or deques) each of length 10000. An element in the i-th deque is a tuple, called a transition, consisting of the state of the particular game at the agent’s i-th turn, the action that the agent took at that state, the reward the agent received for taking the action, and the state that followed the agent’s action.

In [4]:
Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        # save a transition
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

# Q network

![hi](./figures/network-input-output.png)

In [5]:
class DQN(nn.Module):

    # n_input: the current state
    #  (1x52)    +  (56x52)       +       (1x52): the current state
    #    ^hand      ^who plays each card  ^cards not seen yet
    #                       + cards played
    # n_output: probability of playing each card
    #   (1x52)
    def __init__(self, n_input, n_output):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_input, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_output)

    def forward(self, x):
        if x.dtype == torch.float32:
            x = F.relu(self.layer1(x))
            x = F.relu(self.layer2(x))
        else:
            x=x.to(torch.float32)
            x = F.relu(self.layer1(x))
            x = F.relu(self.layer2(x))
        return self.layer3(x)

# Training

In [7]:
# if GPU is to be used
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)
if torch.cuda.is_available():
    print("CUDA is available. GPU can be used.")
    print(f"Device name: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available. Using CPU.")


CUDA is not available. Using CPU.


### Hyperparameters and initializing the networks

In [10]:
EPS_START = 1
EPS_END = 0.1
EPS_DECAY = 5000

BATCH_SIZE = 100
MEMORY_SIZE = 10000

# Learning rate of the optimizer
LR = 1e-4

# soft update rate
TAU = 0.005

# future discount
GAMMA = 0.5


env = Card_Env()
state = env.game.get_network_input()

n_input = len(state)
n_actions = 52

policy_net = DQN(n_input, n_actions).to(device)
# use a target network to prevent oscillation or divergence
target_net = DQN(n_input, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())
# target_net.load_state_dict(torch.load('ev_working_function.pth'))


optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = {turn: ReplayMemory(MEMORY_SIZE) for turn in range(13)}

steps_done = 0

foreign_weight = 'weights/ev_q_function_output_outside_folder.pth'

foreign_net = DQN(n_input, n_actions).to(device)
if device == torch.device('cpu'):
    foreign_net.load_state_dict(torch.load(foreign_weight, map_location=torch.device('cpu')))
else:
    foreign_net.load_state_dict(torch.load(foreign_weight))

env_against_net = Card_Env(foreign_policy= lambda game : net_policy(foreign_net, game))

episode_durations = []
episode_rewards = []


### The $\epsilon$-greedy policy used when training

In [12]:
'''
Given the game state, select an action by the epsilon-greedy policy
'''
def select_action(game):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1

    # epsilon-greedy choice

    if sample > eps_threshold:
        with torch.no_grad():
            # return the index of the card with highest probability
            # predicted from the policy net
            return policy_legal_move(policy_net, game.get_network_input())
    else:
        # random select a legal action
        return torch.tensor([[game.sample_legal_move()]], device=device, dtype=torch.long)


In [ ]:
def plot_durations_rewards(show_result=False):
    fig, ax = plt.subplots(2, 1)

    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    rewards_t = torch.tensor(episode_rewards, dtype=torch.float)

    title = 'Result' if show_result else 'Training...'
    ax[0].set_title(f'{title} Duations each Episode')
    ax[0].set_xlabel('Episode')
    ax[0].set_ylabel('Duration')

    ax[1].set_title(f'{title} Rewards each Episode')
    ax[1].set_xlabel('Episode')
    ax[1].set_ylabel('Reward')

    ax[0].plot(durations_t.numpy())
    ax[1].plot(rewards_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        ax[0].plot(means.numpy())

    if len(rewards_t) >= 100:
        means = rewards_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        ax[1].plot(means.numpy())

    fig.tight_layout()

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        if not show_result:
            display.clear_output(wait=True)
            display.display(fig)
        else:
            display.display(fig)

### A single step optimization of the model using Deep Q-Learning
1) samples a batch from memory, concatenates all the tensors into a single one
2) computes $Q(s_t, a_t)$ and $V(s_{t+1}) = \max_a Q(s_{t+1}, a)$, where $s_t$ --($a_t$)--> $s_{t+1}$
3) computes the loss
4) updates the target network (which is computing $V(s_{t+1}))$ at every step with soft update

In [16]:
def optimize_model():
    transitions = []
    for turn, mem in memory.items():
        if len(mem) >= BATCH_SIZE:
            transitions += mem.sample(BATCH_SIZE)
    if transitions == []:
        return

    # transitions = memory.sample(BATCH_SIZE)
    batch = Transition(*zip(*transitions))

    # mask the non final states and find the corresponding next states
    # We need an illegal move to be a non-final state
    # Right now, we are throwing out all the final states which include the case when
    # the agent ends the game prematurely after playing an illegal move


    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    # print('non_final_mask is', non_final_mask)
    non_final_next_states = [s for s in batch.next_state if s is not None]

    non_final_next_states = torch.cat(non_final_next_states)
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # print('reward_batch is', reward_batch)


    # compute Q(s_t, a)
    # for each state in the batch, find the value of the corresponding action
    state_action_values = policy_net(state_batch.to(torch.float)).gather(1, action_batch)

    # compute V(s_{t+1}) = max_a Q(s_{t+1}, a) for the next_states using the target_net
    next_state_values = torch.zeros(len(transitions), device=device)
    # next_state_values = -10.0 * torch.ones(len(transitions), device=device)

    if non_final_next_states != []:

        with torch.no_grad():
            # print(non_final_mask.shape, target_net(non_final_next_states).max(1).values)
            next_state_values[non_final_mask] = target_net(non_final_next_states).max(1).values  # these will all be negative!

    # R + \gamma max_a Q(s', a)
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # compute the Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # back propagate
    optimizer.zero_grad()
    loss.backward()

    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()


In [9]:
def move_to_card(move):
    return ['2','3','4','5','6','7','8','9','10','J','Q','K','A'][move % 13], ['C', 'D', 'H', 'S'][int(move / 13)]

def simulate_game(policy, verbose=False, from_move=0):
    with torch.no_grad():
        moves_played = 0
        active_player = from_move % 4
        if verbose:
            print(f"Starting new game as player {active_player} from turn {from_move}.")
        test_game = Card_Game()
        for turn in range(52):
            if test_game.current_player != active_player or turn < from_move:
                move = test_game.sample_legal_move()
            else:
                # print(policy_net(test_game.get_network_input().to(device)))
                moves_played += 1
                move = policy_legal_move(policy_net, test_game.get_network_input()).item()
                # move = policy_net(test_game.get_network_input().to(device)).argmax().item()
                if not(move in test_game.get_legal_moves()):
                    if verbose:
                        print(f"Tried to play illegal move {move_to_card(move)}")
                    return moves_played
            if verbose:
                print(f"Player {test_game.current_player} plays {move_to_card(move)}")
            test_game.play_card(move)
            if turn % 4 == 3 and verbose:
                print()
        return moves_played+1


### The main training loop

In [18]:
num_episodes = 5000
update_ind = 100

save_ind = 1000

rewards_list = []
benchmark = (-13, 0)

for i_episode in count():
    # if i_episode % save_ind == 0 and i_episode != 0:
    #     torch.save(policy_net.state_dict(), f'/content/drive/MyDrive/deep-learning/dqn/q_fn-{i_episode}.pth')

    if i_episode % update_ind == 0 and i_episode != 0:
        plot_durations_rewards()
        print(f"Trained {i_episode} episodes")
        avg_reward = sum(rewards_list) / len(rewards_list)
        print(f'Average reward per move: {avg_reward}.')
        simul_results = []
        simul_dist = [0 for i in range(14)]
        for g in range(100):
            res = simulate_game(policy_net, verbose=False, from_move=random.randint(0,3))
            simul_results.append(res)
            simul_dist[res-1] += 1
        print(f"Average simulated game duration: {sum(simul_results) / 100}")
        print(f"Distribution of simulated game lengths: {simul_dist}")
        print(f"Memory bank: {[len(mem) for i, mem in memory.items()]}")
        if benchmark[0] < avg_reward and i_episode >= num_episodes:
            print("New benchmark set.")
            # torch.save(policy_net.state_dict(), 'weights/ev_q_function_output.pth')
            benchmark = (avg_reward, sum(simul_results) / 100)
        rewards_list = []
        print()


    env_against_net.reset()

    player_ind = random.randint(0, 3)
    while env_against_net.game.current_player != player_ind:
        move = env_against_net.game.sample_legal_move()
        env_against_net.game.play_card(move)

    current_reward = 0
    state = torch.tensor(env_against_net.game.get_network_input(), dtype=torch.float32, device=device).unsqueeze(0)

    for t in count():
        action = select_action(env_against_net.game)
        # Select action based on policy network

        # with torch.no_grad():
        #     q_values = policy_net(state)
        #     action = q_values.max(1)[1].view(1, 1)

        # Perform action in the environment
        observation, reward, terminated = env_against_net.step(action.item())
        rewards_list.append(reward)
        reward = torch.tensor([reward], device=device)
        done = terminated

        current_reward += reward

        # Compute next state
        if not terminated:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)
        else:
            next_state = None

        # Store transition in memory
        # int(env.game.turn_counter / 4)
        memory[t].push(state, action, next_state, reward)

        # Move to next state
        state = next_state

        # Perform optimization step
        optimize_model()

        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)

        if done:
            # print(f'Episode {i_episode} ended in {t} steps.')
            episode_durations.append(t+1)
            episode_rewards.append(current_reward)
            break

plot_durations_rewards(show_result=True)

/home/txmszlou/Work/erdos-workshop/deep-learning/dqn-trick-game/utils/card_engine.py:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  card = torch.nn.functional.one_hot(torch.tensor(deck_index, dtype=int), self.num_cards * self.num_players).flatten()
/tmp/ipykernel_3706547/2966759183.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state = torch.tensor(env_against_net.game.get_network_input(), dtype=torch.float32, device=device).unsqueeze(0)
/tmp/ipykernel_3706547/2966759183.py:63: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  next_s

KeyboardInterrupt: 

In [ ]:
optimize_model()

type(batch.state) = <class 'tuple'>
state_batch.shape = torch.Size([100, 3016])


RuntimeError: unflatten: Provided sizes [52, 56] don't multiply up to the size of dim 0 (48) in the input tensor

In [ ]:
# simulate_game(policy_net, verbose=True, from_move=random.randint(0,3))
test = DQN(n_input, n_actions).to(device)
test.load_state_dict(torch.load('weights/ev_q_function_output.pth'))
simul_test = []
simul_policy = []
with torch.no_grad():
    test_dist = [0 for i in range(14)]
    policy_dist = [0 for i in range(14)]
    for g in range(10000):
        res_policy = simulate_game(policy_net, verbose=False, from_move=random.randint(0,3))
        res_test = simulate_game(policy_net, verbose=False, from_move=random.randint(0,3))
        simul_test.append(res_test)
        simul_policy.append(res_policy)
        test_dist[res_test-1] += 1
        policy_dist[res_policy-1] += 1
    print(f"Average simulated game duration of test: {sum(simul_test) / 10000}")
    print(f"Average simulated game duration of policy: {sum(simul_policy) / 10000}")
    print()
    print(f"Distribution of game lengths of test: {test_dist}")
    print(f"Distribution of game lengths of policy: {policy_dist}")


Average simulated game duration of test: 11.1076
Average simulated game duration of policy: 11.1212

Distribution of game lengths of test: [70, 94, 230, 297, 375, 404, 530, 614, 600, 498, 471, 330, 320, 5167]
Distribution of game lengths of policy: [77, 84, 225, 288, 416, 401, 545, 601, 540, 501, 459, 314, 342, 5207]


In [ ]:
with torch.no_grad():
    env.game.reset()
    net_input = env.game.get_network_input().to(device)
    hand = net_input[:52]
    cards = torch.arange(52).to(device)
    q_func_policy = policy_net(net_input)
    # q_func_test = test(net_input)
    print(f'{q_func_policy.argmax()}')
    print(torch.stack((cards, hand, q_func_policy)).transpose(0, 1))

23
tensor([[ 0.0000e+00,  1.0000e+00, -2.0753e+05],
        [ 1.0000e+00,  1.0000e+00, -2.9378e+05],
        [ 2.0000e+00,  0.0000e+00, -2.8098e+05],
        [ 3.0000e+00,  0.0000e+00, -5.0699e+05],
        [ 4.0000e+00,  0.0000e+00, -8.9135e+04],
        [ 5.0000e+00,  0.0000e+00, -2.4677e+05],
        [ 6.0000e+00,  1.0000e+00, -2.0640e+05],
        [ 7.0000e+00,  0.0000e+00, -1.8064e+05],
        [ 8.0000e+00,  0.0000e+00, -5.6643e+05],
        [ 9.0000e+00,  1.0000e+00, -2.5549e+05],
        [ 1.0000e+01,  0.0000e+00, -4.6659e+05],
        [ 1.1000e+01,  0.0000e+00, -6.8289e+05],
        [ 1.2000e+01,  0.0000e+00, -2.1979e+05],
        [ 1.3000e+01,  0.0000e+00, -1.4135e+05],
        [ 1.4000e+01,  1.0000e+00, -3.7770e+05],
        [ 1.5000e+01,  0.0000e+00, -2.6255e+05],
        [ 1.6000e+01,  0.0000e+00, -1.8918e+05],
        [ 1.7000e+01,  1.0000e+00, -1.3708e+05],
        [ 1.8000e+01,  0.0000e+00, -1.7210e+05],
        [ 1.9000e+01,  1.0000e+00, -2.1982e+05],
        [ 2.0000e

In [ ]:
def save_memory(memory_file):
    with open(memory_file, 'wb') as outp:
        pickle.dump(memory, outp, pickle.HIGHEST_PROTOCOL)

save_memory('saved_memory_data.pkl')

KeyboardInterrupt: 

In [ ]:
def load_memory(memory_file):
    with open(memory_file, 'rb') as inp:
        return = pickle.load(inp)

memory = load_memory('saved_memory_data.pkl')

KeyboardInterrupt: 

In [ ]:
torch.save(policy_net.state_dict(), 'ev_q_function_output.pth')

In [ ]:
simulate_game(policy_net, verbose=True, from_move=random.randint(0, 3))

Starting new game as player 0 from turn 0.
Player 0 plays ('J', 'D')
Player 1 plays ('3', 'D')
Player 2 plays ('A', 'D')
Player 3 plays ('2', 'D')

Player 2 plays ('4', 'C')
Player 3 plays ('2', 'C')
Player 0 plays ('J', 'C')
Player 1 plays ('K', 'C')

Player 0 plays ('6', 'C')
Player 1 plays ('Q', 'C')
Player 2 plays ('A', 'C')
Player 3 plays ('10', 'C')

Player 2 plays ('5', 'D')
Player 3 plays ('K', 'D')
Player 0 plays ('6', 'D')
Player 1 plays ('7', 'D')

Player 3 plays ('A', 'H')
Player 0 plays ('6', 'H')
Player 1 plays ('4', 'H')
Player 2 plays ('Q', 'H')

Player 3 plays ('5', 'S')
Player 0 plays ('4', 'S')
Player 1 plays ('8', 'S')
Player 2 plays ('6', 'S')

Player 1 plays ('7', 'S')
Player 2 plays ('10', 'S')
Player 3 plays ('2', 'S')
Player 0 plays ('3', 'S')

Player 2 plays ('9', 'S')
Player 3 plays ('10', 'H')
Player 0 plays ('J', 'S')
Player 1 plays ('K', 'S')

Player 0 plays ('J', 'H')
Player 1 plays ('8', 'C')
Player 2 plays ('7', 'C')
Player 3 plays ('2', 'H')

Player 0 

14